In [10]:
import sqlite3
from cryptography.fernet import Fernet
import secrets
import string
from getpass import getpass
import sys
import tkinter as tk
from tkinter import messagebox
import os,path

# Create & Read file

# Golden KEY

if not os.path.isfile('GoldenKEY.txt'):
    with open('GoldenKEY.txt', 'wb') as GoldenFile:
        KEY = Fernet.generate_key()
        GoldenFile.write(KEY)
    with open('GoldenKEY.txt') as GoldenFile:
        KEY = GoldenFile.read()
else:
    with open('GoldenKEY.txt') as GoldenFile:
        KEY = GoldenFile.read()
    
CodeKEY = Fernet(KEY)

# View generate

service_code 操作選擇  
type_code 分類選擇  

In [11]:
class main_control(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)
        self.title("Account Vault")
        self.geometry("450x650")
        self._frame = None
        self.switch_frame(login_page)

    def switch_frame(self, frame_class, **kwagrs):
        new_frame = frame_class(self, kwagrs)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()
        
        

In [12]:
class login_page(tk.Frame):
    
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Welcome to Account Vault", font=("Times",20)).pack(side="top")
        tk.Label(self, text="Please Enter Password", font=("Times",20)).pack(side="top")
        self.password = tk.Entry(self,show="*")
        self.password.pack()
        tk.Button(self, text="Login", command=self.validate).pack()
    
    def validate(self):
        password = self.password.get()
        #get ultimate password
        demand = "SELECT Password FROM Account_info WHERE Category = 'Ultimate'"
        cursor.execute(demand)
        result = cursor.fetchone()[0]
        decrypted = CodeKEY.decrypt(result.encode())
        
        if password == decrypted.decode():
            self.controler.switch_frame(main_menu)
        else:
            tk.Label(self, text="YOU HAVE NO ACCESS TO THE VAULT !", font=("Times",10)).pack()

In [13]:
class main_menu(tk.Frame):
    
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Welcome Back Master", font=("Times",15)).pack(side="top")
        tk.Label(self, text="What are you looking for today?", font=("Times",20)).pack(side="top")
        tk.Button(self, text="Get Account", command=lambda: self.service_change("1"), height=5, width=20).pack()
        tk.Button(self, text="Add Account", command=lambda: self.service_change("2"), height=5, width=20).pack()
        tk.Button(self, text="Update Account", command=lambda: self.service_change("3"),height=5, width=20).pack()
        #tk.Button(self, text="Exit", command=lambda: self.service_change("4"),height=5, width=20).pack()
    def service_change(self, service_code):
        if service_code == "1" or service_code == "3":
            self.controler.switch_frame(get_type, service_code=service_code)
        elif service_code == "2":
            self.controler.switch_frame(add_account, service_code=service_code)

In [14]:
class get_type(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        if kwargs["service_code"] == "1":
            tk.Label(self, text="Get Account", font=("Times",20)).pack(side="top")
        elif kwargs["service_code"] == "3":
            tk.Label(self, text="Update Account", font=("Times",20)).pack(side="top")
        tk.Label(self, text="What Type of Account Are You Looking For?", font=("Times",15)).pack(side="top")
        
        tk.Button(self, text="GAME", height=5, width=20, command=lambda: self.type_change("1", kwargs["service_code"])).pack()
        tk.Button(self, text="SHOP", height=5, width=20, command=lambda: self.type_change("2", kwargs["service_code"])).pack()
        tk.Button(self, text="WORK", height=5, width=20, command=lambda: self.type_change("3", kwargs["service_code"])).pack()
        tk.Button(self, text="STUDY", height=5, width=20, command=lambda: self.type_change("4", kwargs["service_code"])).pack()
        tk.Button(self, text="BACK TO MAIN MENU", height=5, width=20, command=lambda: self.controler.switch_frame(main_menu)).pack()
    def type_change(self, type_code, service_code):
        if service_code == "1" or service_code == "3":
            self.controler.switch_frame(get_account_serive, type_code=type_code ,service_code=service_code)
            
        
class get_account_serive(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        if kwargs['type_code'] == "1":
            tk.Label(self, text="GAME", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "2":
            tk.Label(self, text="SHOP", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "3":
            tk.Label(self, text="WORK", font=("Times",20)).grid(row=0,column=0)
        elif kwargs['type_code'] == "4":
            tk.Label(self, text="STUDY", font=("Times",20)).grid(row=0,column=0)
            
        tk.Label(self, text="Institutions in This Category", font=("Times",15)).grid(row=1,column=0)
        #organize frame for button
        institution_frame = tk.Frame(self)
        institution_frame.grid(row=3,column=0)
        # get insitutions
        demand = f"SELECT Institution FROM Account_info WHERE Category = {kwargs['type_code']}"
        cursor.execute(demand)
        result = cursor.fetchall()
        result = set(result)
        #organize button 10 in one column
        row=0
        column=0
        for item in result:
            tk.Button(institution_frame, text=item[0], height=2, width=15, 
                      command=lambda item=item: self.show_account(item[0],kwargs['type_code'],kwargs['service_code'])).grid(row=row,column=column)
            row+=1
            if row % 10 ==0:
                column+=1
                row = 0
        if kwargs["service_code"] == "1":
            tk.Button(self, text="Back to Last Page", height=2, width=15, 
                      command=lambda: self.controler.switch_frame(get_type, service_code="1")).grid(row=4, column=0)
        elif kwargs["service_code"] == "3":
            tk.Button(self, text="Back to Last Page", height=2, width=15, 
                      command=lambda: self.controler.switch_frame(get_type, service_code="3")).grid(row=4, column=0)
        
    def show_account(self, institution, type_code, service_code):
        self.controler.switch_frame(show_all_account, ins=institution, type_code=type_code, service_code=service_code)

In [15]:
class show_all_account(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        institution = kwargs['ins']
        tk.Label(self, text=f"Account in {institution}", font=("Times",20)).grid(row=0,column=0)
        #organize button frame
        account_frame = tk.Frame(self)
        account_frame.grid(row=1,column=0)
        #get all account
        institution = "'" + kwargs['ins'] + "'"
        demand = f'SELECT Account FROM Account_info WHERE Institution = {institution}'
        cursor.execute(demand)
        result = cursor.fetchall()
        #get info
        if kwargs["service_code"] == "1":
            #organize button
            row=0
            column=0
            for item in result:
                tk.Button(account_frame, text=item[0], height=2, width=15, 
                          command=lambda account=item[0], institution=institution: self.get_info(account,institution)).grid(row=row,column=column)
                row+=1
                if row % 10 ==0:
                    column+=1
                    row = 0
        #update Account
        elif kwargs["service_code"] == "3":
            #organize button
            row=0
            column=0
            for item in result:
                tk.Button(account_frame, text=item[0], height=2, width=15, 
                          command=lambda account=item[0], 
                          institution=institution: self.controler.switch_frame(update_account, 
                                                                               account=account, 
                                                                              ins = kwargs['ins'],
                                                                              type_code = kwargs["type_code"],
                                                                              service_code = kwargs["service_code"])).grid(row=row,column=column)
                row+=1
                if row % 10 ==0:
                    column+=1
                    row = 0
                    
        tk.Button(self, text="Back to Last Page", height=2, width=15, 
                  command=lambda: self.controler.switch_frame(get_account_serive, 
                                                              type_code=kwargs["type_code"], 
                                                              service_code = kwargs["service_code"])).grid(row=4, column=0)
    def get_info(self, account, institution):
        account = "'" + account + "'"
        demand = f'SELECT Account, Password FROM Account_info WHERE Account = {account} AND Institution = {institution}'
        cursor.execute(demand)
        result = cursor.fetchone()
        password = CodeKEY.decrypt(result[1].encode()).decode()
        
        messagebox.showinfo("Account Info", f"Account: {result[0]}\nPassword: {password}")

In [16]:
class add_account(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        tk.Label(self, text="Add Account", font=("Times",20)).grid(row=0,column=0)
        self.entry_frame = tk.Frame(self)
        self.entry_frame.grid(row=2, column=0)
        
        tk.Label(self.entry_frame, text="INSTITUTION", font=("Times",15)).grid(row=0, column=0)
        self.ins_entry = tk.Entry(self.entry_frame)
        self.ins_entry.grid(row=0,column=1)
        
        tk.Label(self.entry_frame, text="ACCOUNT", font=("Times",15)).grid(row=1, column=0)
        self.account_entry = tk.Entry(self.entry_frame)
        self.account_entry.grid(row=1,column=1)
        
        tk.Label(self.entry_frame, text="PASSWORD", font=("Times",15)).grid(row=2, column=0)
        self.password_entry = tk.Entry(self.entry_frame, show="*")
        self.password_entry.grid(row=2,column=1)
        tk.Button(self.entry_frame, text="Generate", command=self.generate_password).grid(row=2, column=2)
        
        tk.Label(self.entry_frame, text="PASSWORD AGAIN", font=("Times",15)).grid(row=3, column=0)
        self.repassword_entry = tk.Entry(self.entry_frame, show="*")
        self.repassword_entry.grid(row=3,column=1)
        
        tk.Label(self.entry_frame, text="CATEGORY", font=("Times",15)).grid(row=4, column=0)
        category_frame = tk.Frame(self.entry_frame)
        category_frame.grid(row=4, column=1)
        self.category = tk.StringVar()
        
        tk.Radiobutton(category_frame, text="GAME", variable=self.category, value="1").grid(row=0,column=0)
        tk.Radiobutton(category_frame, text="SHOP", variable=self.category, value="2").grid(row=1,column=0)
        tk.Radiobutton(category_frame, text="WORK", variable=self.category, value="3").grid(row=2,column=0)
        tk.Radiobutton(category_frame, text="STUDY", variable=self.category, value="4").grid(row=3,column=0)
        
        tk.Button(self.entry_frame, text="ADD AACOUNT", height=2, width=10, command=self.upload_account).grid(row=5, column=1)
        tk.Button(self.entry_frame, text="Back to Last Page", height=2, width=10, 
                  command=lambda: self.controler.switch_frame(main_menu)).grid(row=6, column=1)
    def generate_password(self):
        self.password_entry.delete(0,tk.END)
        self.repassword_entry.delete(0,tk.END)
        # generate password
        alphabet = string.ascii_letters + string.digits
        while True:
            self.gen_password = ''.join(secrets.choice(alphabet) for i in range(15))
            if (any(c.islower() for c in self.gen_password)
                    and any(c.isupper() for c in self.gen_password)
                    and sum(c.isdigit() for c in self.gen_password) >= 3):
                break
        self.password_entry.insert(0,self.gen_password)
        self.repassword_entry.insert(0,self.gen_password)
        messagebox.showinfo("High Level Password", f"Your Password: {self.gen_password}")
        
    def upload_account(self):
        if self.password_entry.get() != self.repassword_entry.get():
            messagebox.showinfo("WARN","PLEASE CHECK THE PASSWORD")
        elif (self.password_entry.get() == None or self.repassword_entry.get() == None 
              or self.ins_entry.get() == None or self.account_entry.get() == None or self.category.get() == ""):
            messagebox.showinfo("WARN","PLEASE FILL ALL THE BOX")
        else:
            ins = "'" + self.ins_entry.get() + "'"
            password = CodeKEY.encrypt(bytes(self.password_entry.get(),'utf-8')).decode()
            password = "'" + password + "'"
            account = "'" + self.account_entry.get() + "'"
            category = "'" + self.category.get() + "'"
            demand = f"INSERT INTO Account_info (Institution, Account, Password, Category) VALUES ({ins}, {account}, {password}, {category})"
            cursor.execute(demand)
            connection.commit()
            messagebox.showinfo("Inform", "Account Added")
            self.controler.switch_frame(main_menu)
            
            
            
            

In [17]:
class update_account(tk.Frame):
    def __init__(self, controler, kwargs):
        tk.Frame.__init__(self, controler)
        self.controler = controler
        self.kwargs = kwargs
        tk.Label(self, text=f"Update Account", font=("Times",20)).pack(side="top")
        entry_frame = tk.Frame(self)
        entry_frame.pack(side="top")
        
        tk.Label(entry_frame, text="Account", font=("Times",15)).grid(row=0,column=0)
        self.acc_entry = tk.Entry(entry_frame)
        self.acc_entry.grid(row=0, column=1)
        tk.Button(entry_frame, text="Update Account", command=self.update_account).grid(row=1, column=1)
        
        tk.Label(entry_frame, text="Password", font=("Times",15)).grid(row=2,column=0)
        self.pass_entry = tk.Entry(entry_frame,show="*")
        self.pass_entry.grid(row=2, column=1)
        tk.Label(entry_frame, text="Password Again", font=("Times",15)).grid(row=3,column=0)
        self.repass_entry = tk.Entry(entry_frame,show="*")
        self.repass_entry.grid(row=3, column=1)
        tk.Button(entry_frame, text="Update Password", command=self.update_password).grid(row=4, column=1)
        tk.Button(self, text = "Back To Last Page", width=8,height=1, command=lambda:self.controler.switch_frame(show_all_account, type_code=self.kwargs["type_code"], 
                                        service_code=self.kwargs["service_code"], ins=self.kwargs["ins"])).pack(side="bottom")
        tk.Button(self, text = "Delete Account", width=8,height=1, command=self.delete_account).pack(side="bottom")
    
    def update_account(self):
        if self.acc_entry.get() == None:
            messagebox.showinfo("Warn", "Please enter New Account")
        else:
            old_acc = "'" + self.kwargs["account"] + "'"
            ins = "'" + self.kwargs["ins"] + "'"
            new_acc = "'" + self.acc_entry.get() + "'"
            demand = f"UPDATE Account_info SET Account = {new_acc} WHERE Account = {old_acc} AND Institution = {ins}"
            cursor.execute(demand)
            connection.commit()
            messagebox.showinfo("Succeed", "Account Updated")
            self.controler.switch_frame(show_all_account, 
                                        type_code=self.kwargs["type_code"], 
                                        service_code=self.kwargs["service_code"], 
                                        ins=self.kwargs["ins"])
    def update_password(self):
        if self.pass_entry.get() == None or self.repass_entry.get() == None:
            messagebox.showinfo("Warn", "Please enter New Password")
        elif self.pass_entry.get() != self.repass_entry.get():
            messagebox.showinfo("Warn", "Please Check New Password")
        else:
            acc = "'" + self.kwargs["account"] + "'"
            ins = "'" + self.kwargs["ins"] + "'"
            new_pass = "'" + CodeKEY.encrypt(bytes(self.pass_entry.get(),'utf-8')).decode() + "'"
            demand = f"UPDATE Account_info SET Password = {new_pass} WHERE Account = {acc} AND Institution = {ins}"
            cursor.execute(demand)
            connection.commit()
            messagebox.showinfo("Succeed", "Password Updated")
            self.controler.switch_frame(show_all_account, 
                                        type_code=self.kwargs["type_code"], 
                                        service_code=self.kwargs["service_code"], 
                                        ins=self.kwargs["ins"]) 
    def delete_account(self):
        if messagebox.askquestion("Warn", "Are You Sure Want to DELETE Account?"):
            acc = "'" + self.kwargs["account"] + "'"
            ins = "'" + self.kwargs["ins"] + "'"
            demand = f"DELETE FROM Account_info WHERE Account = {acc} AND Institution = {ins}"
            cursor.execute(demand)
            connection.commit()
            messagebox.showinfo("Succeed", "Account Deleted")
            self.controler.switch_frame(show_all_account, 
                                        type_code=self.kwargs["type_code"], 
                                        service_code=self.kwargs["service_code"], 
                                        ins=self.kwargs["ins"]) 

In [18]:
if os.path.isfile('Account_data.db') == False or os.path.isfile('GoldenKEY.txt') == False:
    
    if not os.path.isfile('Account_data.db'):
                connection = sqlite3.connect('Account_data.db')
                cursor = connection.cursor()
                demand = "CREATE TABLE Account_info (DateTime DATETIME CURRENT_TIMESTAMP, Institution VARCHAR(50) NOT NULL, Account VARCHAR(50) NOT NULL, Password VARCHAR(100) NOT NULL, Category VARCHAR(50) NOT NULL, PRIMARY KEY (DateTime));"
                cursor.execute(demand)
                connection.commit()
    else:
        connection = sqlite3.connect('Account_data.db')
        cursor = connection.cursor()
        
    def register(password, repassword):
        global CodeKEY
        
        if password != repassword:
            messagebox.showwarning("WARN", "Please CHECK The Password")
        else:
            if not os.path.isfile('GoldenKEY.txt'):
                with open('GoldenKEY.txt', 'wb') as GoldenFile:
                    KEY = Fernet.generate_key()
                    GoldenFile.write(KEY)
                with open('GoldenKEY.txt') as GoldenFile:
                    KEY = GoldenFile.read()
            else:
                with open('GoldenKEY.txt') as GoldenFile:
                    KEY = GoldenFile.read()

            CodeKEY = Fernet(KEY)

            ULT = "'Ultimate'"
            Password_encrypted = CodeKEY.encrypt(bytes(password, 'utf-8')).decode()
            Password_encrypted = "'" + Password_encrypted + "'"
            demand = f"INSERT INTO Account_info (Institution, Account, Password, Category) VALUES ({ULT}, {ULT}, {Password_encrypted}, {ULT})"
            cursor.execute(demand)
            connection.commit()
            messagebox.showinfo("Welcome", "Register Succeed")
            register_window.destroy()
            
            
    register_window = tk.Tk()
    register_window.geometry("570x150")
    register_window.title("Register")
    reg_frame = tk.Frame(register_window)
    reg_frame.pack(side='top')
    tk.Label(reg_frame, text="PLEASE CREATE ULTIMATE PASSWORD", font=("Times", 20)).grid(row=0, column=1)
    tk.Label(reg_frame, text="PASSWORD", font=("Times", 20)).grid(row=1, column=0)
    reg_pass = tk.Entry(reg_frame, show="*")
    reg_pass.grid(row=1, column=1)
    tk.Label(reg_frame, text="PASSWORD AGAIN", font=("Times", 20)).grid(row=2, column=0)
    reg_repass = tk.Entry(reg_frame, show="*")
    reg_repass.grid(row=2, column=1)
    
    tk.Button(reg_frame, text="REGISTER", command=lambda: register(reg_pass.get(),reg_repass.get())).grid(row=3, column=1)
    register_window.mainloop()
    
else:
    connection = sqlite3.connect('Account_data.db')
    cursor = connection.cursor()
    with open('GoldenKEY.txt') as GoldenFile:
        KEY = GoldenFile.read()
    CodeKEY = Fernet(KEY)
    
app = main_control()
app.mainloop()

